In [16]:
# Base utilities
import os

import googlemaps
from datetime import datetime

# Data Mining
import math
import random
random_state = 42
random.seed(random_state)
seed=random_state
import numpy as np
import pandas as pd
import geopandas as gpd
# import osmnx as ox
import pandana as pdn
import pickle as pkl
import folium

# WKT to Lat and Long

from shapely import wkt
from shapely.geometry import Point

# Plot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn as sns

# Learning
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, train_test_split, KFold

from sklearn.preprocessing import OneHotEncoder

# Models
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from scipy.spatial import distance

# Directories
for d in ["data", "models", "logs", "results"]:
    if not os.path.isdir(d):
        os.mkdir(d)

Bring the buildings

In [17]:
# Bring buildings

gdf = gpd.read_file("../input_data/buildings_with_section/buildings_w_section.shp")
crs_objetivo = 'EPSG:4326'  # WGS84
gdf = gdf.to_crs(crs_objetivo)

gdf['Centroide'] = gdf['geometry'].centroid

gdf_viviendas = gdf[(gdf['Vivienda'] != 0) | (gdf['Duplex'] != 0)]
gdf_oficinas = gdf[(gdf['Oficina'] != 0)]

gdf_viviendas = gdf_viviendas[['Town', 'Centroide', 'Percentil']]
gdf_viviendas['D_long'] = gdf_viviendas['Centroide'].apply(lambda p: p.x)
gdf_viviendas['D_lat'] = gdf_viviendas['Centroide'].apply(lambda p: p.y)
# gdf_viviendas = gdf_viviendas.drop(columns=['Centroide']).reset_index(drop=True)

gdf_oficinas = gdf_oficinas[['Town', 'Centroide']]
gdf_oficinas['D_long'] = gdf_oficinas['Centroide'].apply(lambda p: p.x)
gdf_oficinas['D_lat'] = gdf_oficinas['Centroide'].apply(lambda p: p.y)
gdf_oficinas = gdf_oficinas.drop(columns=['Centroide']).reset_index(drop=True)

C:\Users\iazka\AppData\Local\Temp\ipykernel_11516\1155712968.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Centroide'] = gdf['geometry'].centroid


In [18]:
# PLOT ON A FOLIUM MAP

map = folium.Map(location=[43.125654, -2.237611], zoom_start=8)

# Add points
for _, row in gdf_viviendas.iterrows():
    folium.Marker(
        location=[row['D_lat'], row['D_long']],
        popup=row['Town'],  # Optional: to show the town name on click
    ).add_to(map)

map.save("../input_data/buildings_with_section/map_viviendas_100.html")

map = folium.Map(location=[43.125654, -2.237611], zoom_start=8)

# Add points
for _, row in gdf_oficinas.iterrows():
    folium.Marker(
        location=[row['D_lat'], row['D_long']],
        popup=row['Town'],  # Optional: to show the town name on click
    ).add_to(map)

# map.save("../input_data/buildings_with_section/map_oficinas_100.html")

## ORIGIN - RESIDENTIAL HOUSING

In [19]:
# We think that the survey was done to "typical" people, not people that live in baserris. We delete the external 10%
X = gdf_viviendas[['D_lat', 'D_long']]

# Initialize NearestNeighbors
neighbors = NearestNeighbors(n_neighbors=50)  
neighbors.fit(X)

distances, indices = neighbors.kneighbors(X)

# We exclude the first column because it's the distance to itself (0.0)
average_distances = np.mean(distances[:, 1:], axis=1)

threshold = np.percentile(average_distances, 80) 

points_to_remove = np.where(average_distances >= threshold)[0]

gdf_viviendas = gdf_viviendas.drop(gdf_viviendas.index[points_to_remove]).reset_index(drop=True)

In [20]:
# There are too many origins, to reduce computing time, I get rid of 50%.

def retain_60_percent(group):
    return group.sample(frac=0.5)

# Group by 'town', apply the function, and concatenate the results
gdf_viviendas_use = gdf_viviendas.groupby('Town').apply(retain_60_percent).reset_index(drop=True)

In [21]:
map = folium.Map(location=[43.125654, -2.237611], zoom_start=11)

# Add points
for _, row in gdf_viviendas_use.iterrows():
    folium.Marker(
        location=[row['D_lat'], row['D_long']],
        popup=row['Town'],  # Optional: to show the town name on click
    ).add_to(map)

# map.save("../input_data/buildings_with_section/map_viviendas_use.html")

## DESTINATION - OFFICES

In [22]:
# The problem is that if a office is the furthest in a town (Igeldo), it might have been chosen many times nonsense. We want to choose offices from a place that are offices commonly, not a office by coincidence because otherwise bus communication can be very bad and the model has bad behavior.
X = gdf_oficinas[['D_lat', 'D_long']]

# Initialize NearestNeighbors
neighbors = NearestNeighbors(n_neighbors=6)  # Using 6 because the point itself is included
neighbors.fit(X)

# Calculate the distances and indices of the 5 nearest neighbors for each point
distances, indices = neighbors.kneighbors(X)

# Calculate the average distance to the 5 nearest neighbors for each point
# We exclude the first column because it's the distance to itself (0.0)
average_distances = np.mean(distances[:, 1:], axis=1)

threshold = np.percentile(average_distances, 95) #5%

points_to_remove = np.where(average_distances >= threshold)[0]

gdf_oficinas_95 = gdf_oficinas.drop(gdf_oficinas.index[points_to_remove]).reset_index(drop=True)

In [23]:
# I tried to calculate the number of clusters taking into account not just the number of buildings of each town, but also the dispersion of them. 
# The result was not good.

# std_dev = gdf_oficinas_95.groupby('Town').agg({'D_lat': np.std, 'D_long': np.std})
# std_dev['dispersión'] = std_dev['D_lat'] + std_dev['D_long']

# num_edificios_por_pueblo = gdf_oficinas_95['Town'].value_counts()
# diferencia = num_edificios_por_pueblo.max() - num_edificios_por_pueblo.min()
# num_por_pueblo = num_edificios_por_pueblo / diferencia

# num_por_pueblo_df = num_por_pueblo.reset_index()
# num_por_pueblo_df.columns = ['Town', 'num_por_pueblo']

# # Unimos los dos DataFrames
# resultado = pd.merge(std_dev.reset_index(), num_por_pueblo_df, on='Town')

# # Calculamos la multiplicación de num_por_pueblo por la dispersión
# resultado['metrica_final'] = resultado['num_por_pueblo'] * resultado['dispersión']

# # Ordenamos el resultado
# resultado_ordenado = resultado[['Town', 'metrica_final']].sort_values(by='metrica_final', ascending=False)
# resultado_ordenado['metrica_final'] = resultado_ordenado['metrica_final']*2500

# print(resultado_ordenado)

In [24]:
# Number of clusters per town

conteo_edificios = gdf_oficinas_95.groupby('Town').size()

def determinar_valor(num_edificios, pueblo):
    if pueblo == 'Donostia/San Sebastian':
        return 25
    elif pueblo == 'Irun':
        return 15
    elif num_edificios < 40:
        return 8
    else:
        return 12

pueblos = []
valores = []
for pueblo, num_edificios in conteo_edificios.items():
    pueblos.append(pueblo)
    valores.append(determinar_valor(num_edificios, pueblo))

num_clusters = pd.DataFrame({
    'Town': pueblos,
    'num_clusters': valores
})

In [25]:
# CREATE CLUSTERS

colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
           'gray', 'black', 'lightgray']

# Crear un mapa base
mapa = folium.Map(location=[gdf_oficinas_95['D_lat'].mean(), gdf_oficinas_95['D_long'].mean()], zoom_start=10)

gdf_oficinas_95 = pd.merge(gdf_oficinas_95, num_clusters, on='Town', how='left')

# Iterar sobre cada pueblo y realizar la clusterización
for pueblo in gdf_oficinas_95['Town'].unique():
    datos_pueblo = gdf_oficinas_95[gdf_oficinas_95['Town'] == pueblo][['D_lat', 'D_long']]
    num_clusters = gdf_oficinas_95[gdf_oficinas_95['Town'] == pueblo]['num_clusters'].iloc[0]
    kmeans = KMeans(n_clusters=num_clusters).fit(datos_pueblo)
    cluster_labels = kmeans.labels_
    
    # Añadir marcadores de cluster al mapa
    for lat, lon, label in zip(datos_pueblo['D_lat'], datos_pueblo['D_long'], cluster_labels):
        color_cluster = colores[label % len(colores)]  # Asignar color, rotando si hay más clusters que colores
        folium.Marker(
            [lat, lon],
            icon=folium.Icon(color=color_cluster),
            popup=f'Town: {pueblo}, Cluster: {label}'
        ).add_to(mapa)

# Mostrar el mapa
# mapa.save('../input_data/buildings_with_section/mapa_clusters.html')

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\clus

In [26]:
# ASSIGN A REPRESENTATIVE BUILDING TO EACH CLUSTER. THE CLOSEST BUILDING TO THE CENTER OF THE CLUSTER.

# Función para encontrar el punto más cercano al centro de un cluster
def punto_mas_cercano_al_centro(centro, puntos):
    punto_cercano = None
    min_dist = float('inf')
    for punto in puntos.itertuples(index=False):
        # Asegúrate de usar los nombres correctos de tus columnas aquí
        dist = distance.euclidean(centro, (punto.D_lat, punto.D_long))
        if dist < min_dist:
            min_dist = dist
            punto_cercano = punto
    return punto_cercano

# Preparar el segundo mapa para mostrar un edificio representativo por cada cluster
mapa_representativo = folium.Map(location=[gdf_oficinas_95['D_lat'].mean(), gdf_oficinas_95['D_long'].mean()], zoom_start=10)

oficinas_cluster = []

# Realizar la clusterización y encontrar un punto representativo por cluster
for pueblo in gdf_oficinas_95['Town'].unique():
    datos_pueblo = gdf_oficinas_95[gdf_oficinas_95['Town'] == pueblo][['D_lat', 'D_long']]
    num_clusters = gdf_oficinas_95[gdf_oficinas_95['Town'] == pueblo]['num_clusters'].iloc[0]
    
    kmeans = KMeans(n_clusters=num_clusters).fit(datos_pueblo)
    cluster_labels = kmeans.labels_
    centros_clusters = kmeans.cluster_centers_
    
    # Para cada cluster, encontrar el punto más cercano al centro y añadirlo al mapa
    for i, centro in enumerate(centros_clusters):
        puntos_cluster = datos_pueblo.iloc[[label == i for label in cluster_labels]]
        punto_representativo = punto_mas_cercano_al_centro(centro, puntos_cluster)
        folium.Marker(
            [punto_representativo.D_lat, punto_representativo.D_long],
            icon=folium.Icon(color=colores[i % len(colores)], icon='star'),
            popup=f'Town: {pueblo}, Cluster: {i}, Representativo'
        ).add_to(mapa_representativo)
        oficinas_cluster.append({
            'Town': pueblo,
            'D_lat': punto_representativo.D_lat,
            'D_long': punto_representativo.D_long
        })
oficinas_cluster = pd.DataFrame(oficinas_cluster)

# Guardar el mapa de puntos representativos en un archivo HTML
# mapa_representativo.save('../input_data/buildings_with_section/mapa_clusters_representativos.html')

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\clus

## SAVE BOTH DATAFRAMES

In [30]:
# Save
gdf_viviendas_use.to_csv('../input_data/origins.csv', index=True)
oficinas_cluster.to_csv('../input_data/destinations.csv', index=True)

In [31]:
gdf_viviendas_use
# oficinas_cluster

,Town,Centroide,Percentil,D_long,D_lat
0,Andoain,POINT (-2.02048 43.21801),1,-2.020480,43.218011
1,Andoain,POINT (-2.01961 43.21887),3,-2.019607,43.218870
2,Andoain,POINT (-2.02403 43.21271),4,-2.024029,43.212713
3,Andoain,POINT (-2.02070 43.22087),1,-2.020697,43.220865
4,Andoain,POINT (-2.02416 43.22329),1,-2.024157,43.223286
...,...,...,...,...,...
12391,Zumarraga,POINT (-2.30627 43.08699),4,-2.306265,43.086993
12392,Zumarraga,POINT (-2.31216 43.08539),1,-2.312156,43.085393
12393,Zumarraga,POINT (-2.31647 43.08925),3,-2.316473,43.089248
12394,Zumarraga,POINT (-2.31491 43.08254),1,-2.314907,43.082544
